<a href="https://colab.research.google.com/github/RejaulBSSE1324/text2code-seq2seq/blob/main/seq2seq_code_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-Python Code Generation Using Seq2Seq Models

This notebook implements and compares three architectures:
1. Vanilla RNN Seq2Seq
2. LSTM Seq2Seq
3. LSTM with Attention Mechanism

**Author:** [Your Name]  
**Dataset:** CodeSearchNet Python

## 1. Setup and Installations

In [1]:
# Install required packages
!pip install datasets torch torchtext nltk sacrebleu matplotlib seaborn tqdm -q

import nltk
nltk.download('punkt', quiet=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 6.8 MB/s eta 0:00:00


True

In [2]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tqdm.auto import tqdm
import random
import json
from datasets import load_dataset
from sacrebleu.metrics import BLEU

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## 2. Data Loading and Preprocessing

In [3]:
# Load CodeSearchNet dataset
print("Loading dataset...")
dataset = load_dataset("Nan-Do/code-search-net-python", split="train")

# Configuration
MAX_SAMPLES = 10000  # Use 10k samples for training
MAX_DOC_LEN = 50     # Maximum docstring length
MAX_CODE_LEN = 80    # Maximum code length

print(f"Total dataset size: {len(dataset)}")
print(f"Using {MAX_SAMPLES} samples")

Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-ee77a7de79eb2a(…):   0%|          | 0.00/155M [00:00<?, ?B/s]

data/train-00001-of-00004-648b3bede2edf6(…):   0%|          | 0.00/139M [00:00<?, ?B/s]

data/train-00002-of-00004-1dfd72b171e6b2(…):   0%|          | 0.00/153M [00:00<?, ?B/s]

data/train-00003-of-00004-184ab6d0e3c690(…):   0%|          | 0.00/151M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/455243 [00:00<?, ? examples/s]

Total dataset size: 455243
Using 10000 samples


In [4]:
# Examine a sample
sample = dataset[0]
print("Sample docstring:", sample['docstring'][:200])
print("\nSample code:", sample['code'][:200])

Sample docstring: Trains a k-nearest neighbors classifier for face recognition.

    :param train_dir: directory that contains a sub-directory for each known person, with its name.

     (View in source code to see tra

Sample code: def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    """
    Trains a k-nearest neighbors classifier for face recognition.

    :param train_dir: dire


In [5]:
# Vocabulary class
class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.word2idx = {}
        self.idx2word = {}
        self.word2count = Counter()
        self.n_words = 0

        # Special tokens
        self.PAD_token = 0
        self.SOS_token = 1
        self.EOS_token = 2
        self.UNK_token = 3

        self.add_word('<PAD>')
        self.add_word('<SOS>')
        self.add_word('<EOS>')
        self.add_word('<UNK>')

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.n_words
            self.idx2word[self.n_words] = word
            self.n_words += 1
        self.word2count[word] += 1

    def add_sentence(self, sentence):
        for word in sentence.split():
            self.add_word(word)

    def __len__(self):
        return self.n_words

In [6]:
# Preprocessing function
def preprocess_data(dataset, max_samples, max_doc_len, max_code_len):
    """Preprocess dataset and create vocabularies"""

    processed_data = []
    doc_vocab = Vocabulary('docstring')
    code_vocab = Vocabulary('code')

    for i in tqdm(range(min(max_samples, len(dataset))), desc="Processing data"):
        example = dataset[i]

        # Get docstring and code
        docstring = example['docstring'].strip()
        code = example['code'].strip()

        # Skip if empty
        if not docstring or not code:
            continue

        # Simple tokenization (whitespace)
        doc_tokens = docstring.split()
        code_tokens = code.split()

        # Skip if too long
        if len(doc_tokens) > max_doc_len or len(code_tokens) > max_code_len:
            continue

        # Skip if too short
        if len(doc_tokens) < 3 or len(code_tokens) < 3:
            continue

        # Add to vocabularies
        doc_vocab.add_sentence(docstring)
        code_vocab.add_sentence(code)

        processed_data.append({
            'docstring': docstring,
            'code': code
        })

    print(f"\nProcessed {len(processed_data)} valid examples")
    print(f"Docstring vocabulary size: {len(doc_vocab)}")
    print(f"Code vocabulary size: {len(code_vocab)}")

    return processed_data, doc_vocab, code_vocab

# Process the data
processed_data, doc_vocab, code_vocab = preprocess_data(
    dataset, MAX_SAMPLES, MAX_DOC_LEN, MAX_CODE_LEN
)

Processing data:   0%|          | 0/10000 [00:00<?, ?it/s]


Processed 5281 valid examples
Docstring vocabulary size: 13977
Code vocabulary size: 61273


In [7]:
# Split into train/val/test
def split_data(data, train_ratio=0.8, val_ratio=0.1):
    """Split data into train, validation, and test sets"""
    random.shuffle(data)

    n = len(data)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))

    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(processed_data)

print(f"Train: {len(train_data)}")
print(f"Val: {len(val_data)}")
print(f"Test: {len(test_data)}")

Train: 4224
Val: 528
Test: 529


In [8]:
# Dataset class
class CodeDataset(Dataset):
    def __init__(self, data, doc_vocab, code_vocab):
        self.data = data
        self.doc_vocab = doc_vocab
        self.code_vocab = code_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]

        # Convert to indices
        doc_tokens = example['docstring'].split()
        code_tokens = example['code'].split()

        doc_indices = [self.doc_vocab.word2idx.get(w, self.doc_vocab.UNK_token)
                       for w in doc_tokens]
        code_indices = [self.code_vocab.SOS_token] + \
                       [self.code_vocab.word2idx.get(w, self.code_vocab.UNK_token)
                        for w in code_tokens] + \
                       [self.code_vocab.EOS_token]

        return {
            'doc': torch.LongTensor(doc_indices),
            'code': torch.LongTensor(code_indices),
            'doc_text': example['docstring'],
            'code_text': example['code']
        }

# Collate function for batching
def collate_fn(batch):
    doc_batch = [item['doc'] for item in batch]
    code_batch = [item['code'] for item in batch]
    doc_texts = [item['doc_text'] for item in batch]
    code_texts = [item['code_text'] for item in batch]

    # Pad sequences
    doc_padded = pad_sequence(doc_batch, batch_first=True, padding_value=0)
    code_padded = pad_sequence(code_batch, batch_first=True, padding_value=0)

    # Create masks
    doc_lengths = torch.LongTensor([len(d) for d in doc_batch])
    code_lengths = torch.LongTensor([len(c) for c in code_batch])

    return {
        'doc': doc_padded,
        'code': code_padded,
        'doc_lengths': doc_lengths,
        'code_lengths': code_lengths,
        'doc_texts': doc_texts,
        'code_texts': code_texts
    }

# Create datasets
train_dataset = CodeDataset(train_data, doc_vocab, code_vocab)
val_dataset = CodeDataset(val_data, doc_vocab, code_vocab)
test_dataset = CodeDataset(test_data, doc_vocab, code_vocab)

# Create dataloaders
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                         shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                       shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=collate_fn)

print("Data loaders created successfully!")

Data loaders created successfully!


## 3. Model Implementations

### 3.1 Vanilla RNN Seq2Seq

In [9]:
class RNNEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(RNNEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        # Pack padded sequence
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        outputs, hidden = self.rnn(packed)
        # Unpack
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs, hidden

class RNNDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(RNNDecoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output, hidden

class VanillaSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(VanillaSeq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        max_len = tgt.size(1)
        vocab_size = self.decoder.fc.out_features

        # Encode
        _, hidden = self.encoder(src, src_lengths)

        # Decode
        outputs = torch.zeros(batch_size, max_len, vocab_size).to(src.device)
        input_token = tgt[:, 0].unsqueeze(1)  # Start with SOS

        for t in range(1, max_len):
            output, hidden = self.decoder(input_token, hidden)
            outputs[:, t, :] = output.squeeze(1)

            # Teacher forcing
            use_teacher_forcing = random.random() < teacher_forcing_ratio
            if use_teacher_forcing:
                input_token = tgt[:, t].unsqueeze(1)
            else:
                input_token = output.argmax(2)

        return outputs

print("Vanilla RNN Seq2Seq model defined!")

Vanilla RNN Seq2Seq model defined!


### 3.2 LSTM Seq2Seq

In [10]:
class LSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(LSTMEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        outputs, (hidden, cell) = self.lstm(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs, (hidden, cell)

class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(LSTMDecoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output)
        return output, hidden

class LSTMSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(LSTMSeq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        max_len = tgt.size(1)
        vocab_size = self.decoder.fc.out_features

        # Encode
        _, hidden = self.encoder(src, src_lengths)

        # Decode
        outputs = torch.zeros(batch_size, max_len, vocab_size).to(src.device)
        input_token = tgt[:, 0].unsqueeze(1)

        for t in range(1, max_len):
            output, hidden = self.decoder(input_token, hidden)
            outputs[:, t, :] = output.squeeze(1)

            use_teacher_forcing = random.random() < teacher_forcing_ratio
            if use_teacher_forcing:
                input_token = tgt[:, t].unsqueeze(1)
            else:
                input_token = output.argmax(2)

        return outputs

print("LSTM Seq2Seq model defined!")

LSTM Seq2Seq model defined!


### 3.3 LSTM with Attention

In [11]:
class BidirectionalLSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(BidirectionalLSTMEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc_hidden = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc_cell = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        outputs, (hidden, cell) = self.lstm(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        # Combine bidirectional hidden states
        hidden = torch.tanh(self.fc_hidden(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))).unsqueeze(0)
        cell = torch.tanh(self.fc_cell(torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1))).unsqueeze(0)

        return outputs, (hidden, cell)

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim, encoder_dim):
        super(BahdanauAttention, self).__init__()
        self.attn = nn.Linear(hidden_dim + encoder_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs, mask=None):
        # hidden: (batch, hidden_dim)
        # encoder_outputs: (batch, seq_len, encoder_dim)

        batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)

        # Repeat hidden state
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)

        # Concatenate and compute energy
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)

        # Apply mask if provided
        if mask is not None:
            attention = attention.masked_fill(mask == 0, -1e10)

        # Softmax
        attention_weights = torch.softmax(attention, dim=1)

        # Compute context
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)

        return context, attention_weights

class AttentionDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, encoder_dim):
        super(AttentionDecoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.attention = BahdanauAttention(hidden_dim, encoder_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim + encoder_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden, encoder_outputs, mask=None):
        embedded = self.embedding(x)

        # Get attention context
        context, attention_weights = self.attention(
            hidden[0].squeeze(0), encoder_outputs, mask
        )

        # Combine embedding and context
        lstm_input = torch.cat((embedded, context), dim=2)

        # LSTM forward
        output, hidden = self.lstm(lstm_input, hidden)

        # Prediction
        output = self.fc(output)

        return output, hidden, attention_weights

class AttentionSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(AttentionSeq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        max_len = tgt.size(1)
        vocab_size = self.decoder.fc.out_features

        # Create mask for encoder outputs
        mask = torch.zeros(batch_size, src.size(1)).to(src.device)
        for i, length in enumerate(src_lengths):
            mask[i, :length] = 1

        # Encode
        encoder_outputs, hidden = self.encoder(src, src_lengths)

        # Decode
        outputs = torch.zeros(batch_size, max_len, vocab_size).to(src.device)
        attentions = torch.zeros(batch_size, max_len, src.size(1)).to(src.device)
        input_token = tgt[:, 0].unsqueeze(1)

        for t in range(1, max_len):
            output, hidden, attention_weights = self.decoder(
                input_token, hidden, encoder_outputs, mask
            )
            outputs[:, t, :] = output.squeeze(1)
            attentions[:, t, :] = attention_weights

            use_teacher_forcing = random.random() < teacher_forcing_ratio
            if use_teacher_forcing:
                input_token = tgt[:, t].unsqueeze(1)
            else:
                input_token = output.argmax(2)

        return outputs, attentions

print("Attention-based LSTM Seq2Seq model defined!")

Attention-based LSTM Seq2Seq model defined!


## 4. Training Functions

In [12]:
def train_epoch(model, dataloader, optimizer, criterion, device, clip=1.0, is_attention=False):
    """Train for one epoch"""
    model.train()
    epoch_loss = 0

    for batch in tqdm(dataloader, desc="Training", leave=False):
        src = batch['doc'].to(device)
        tgt = batch['code'].to(device)
        src_lengths = batch['doc_lengths']

        optimizer.zero_grad()

        # Forward pass
        if is_attention:
            output, _ = model(src, src_lengths, tgt, teacher_forcing_ratio=0.5)
        else:
            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0.5)

        # Calculate loss (ignore first token which is SOS)
        output = output[:, 1:].reshape(-1, output.shape[-1])
        tgt = tgt[:, 1:].reshape(-1)

        loss = criterion(output, tgt)

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device, is_attention=False):
    """Evaluate model"""
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            src = batch['doc'].to(device)
            tgt = batch['code'].to(device)
            src_lengths = batch['doc_lengths']

            # Forward pass (no teacher forcing during evaluation)
            if is_attention:
                output, _ = model(src, src_lengths, tgt, teacher_forcing_ratio=0)
            else:
                output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)

            # Calculate loss
            output = output[:, 1:].reshape(-1, output.shape[-1])
            tgt = tgt[:, 1:].reshape(-1)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

def train_model(model, train_loader, val_loader, optimizer, criterion,
                n_epochs, device, model_name, is_attention=False):
    """Complete training loop"""
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')

    print(f"\nTraining {model_name}...")

    for epoch in range(n_epochs):
        train_loss = train_epoch(model, train_loader, optimizer, criterion,
                                device, is_attention=is_attention)
        val_loss = evaluate(model, val_loader, criterion, device,
                          is_attention=is_attention)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{n_epochs} | "
              f"Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'{model_name}_best.pt')
            print(f"  --> Saved best model")

    return train_losses, val_losses

print("Training functions defined!")

Training functions defined!


## 5. Train All Models

In [13]:
# Model hyperparameters
EMBED_DIM = 256
HIDDEN_DIM = 256
N_EPOCHS = 10
LEARNING_RATE = 0.001

# Loss function
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding

# Store results
all_results = {}

### 5.1 Train Vanilla RNN

In [ ]:
# Create Vanilla RNN model
rnn_encoder = RNNEncoder(len(doc_vocab), EMBED_DIM, HIDDEN_DIM)
rnn_decoder = RNNDecoder(len(code_vocab), EMBED_DIM, HIDDEN_DIM)
vanilla_model = VanillaSeq2Seq(rnn_encoder, rnn_decoder).to(device)

# Optimizer
vanilla_optimizer = optim.Adam(vanilla_model.parameters(), lr=LEARNING_RATE)

# Train
vanilla_train_losses, vanilla_val_losses = train_model(
    vanilla_model, train_loader, val_loader, vanilla_optimizer, criterion,
    N_EPOCHS, device, 'vanilla_rnn', is_attention=False
)

all_results['Vanilla RNN'] = {
    'train_losses': vanilla_train_losses,
    'val_losses': vanilla_val_losses,
    'model': vanilla_model
}


Training vanilla_rnn...


Training:   0%|          | 0/132 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch 1/10 | Train Loss: 8.8756 | Val Loss: 8.6023
  --> Saved best model


Training:   0%|          | 0/132 [00:00<?, ?it/s]

...

Epoch 3/10 | Train Loss: 7.5795 | Val Loss: 8.9090


Training:   0%|          | 0/132 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch 4/10 | Train Loss: 7.3368 | Val Loss: 8.9996


Training:   0%|          | 0/132 [00:00<?, ?it/s]

### 5.2 Train LSTM

In [ ]:
# Create LSTM model
lstm_encoder = LSTMEncoder(len(doc_vocab), EMBED_DIM, HIDDEN_DIM)
lstm_decoder = LSTMDecoder(len(code_vocab), EMBED_DIM, HIDDEN_DIM)
lstm_model = LSTMSeq2Seq(lstm_encoder, lstm_decoder).to(device)

# Optimizer
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE)

# Train
lstm_train_losses, lstm_val_losses = train_model(
    lstm_model, train_loader, val_loader, lstm_optimizer, criterion,
    N_EPOCHS, device, 'lstm', is_attention=False
)

all_results['LSTM'] = {
    'train_losses': lstm_train_losses,
    'val_losses': lstm_val_losses,
    'model': lstm_model
}

### 5.3 Train LSTM with Attention

In [ ]:
# Create Attention model
attn_encoder = BidirectionalLSTMEncoder(len(doc_vocab), EMBED_DIM, HIDDEN_DIM)
attn_decoder = AttentionDecoder(len(code_vocab), EMBED_DIM, HIDDEN_DIM, HIDDEN_DIM * 2)
attn_model = AttentionSeq2Seq(attn_encoder, attn_decoder).to(device)

# Optimizer
attn_optimizer = optim.Adam(attn_model.parameters(), lr=LEARNING_RATE)

# Train
attn_train_losses, attn_val_losses = train_model(
    attn_model, train_loader, val_loader, attn_optimizer, criterion,
    N_EPOCHS, device, 'lstm_attention', is_attention=True
)

all_results['LSTM + Attention'] = {
    'train_losses': attn_train_losses,
    'val_losses': attn_val_losses,
    'model': attn_model
}

## 6. Visualization of Training Progress

In [ ]:
# Plot training and validation losses
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training loss
for name, results in all_results.items():
    axes[0].plot(results['train_losses'], label=name, marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss Comparison')
axes[0].legend()
axes[0].grid(True)

# Validation loss
for name, results in all_results.items():
    axes[1].plot(results['val_losses'], label=name, marker='o')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Validation Loss Comparison')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training curves saved as 'training_curves.png'")

## 7. Evaluation Metrics

In [ ]:
def generate_code(model, doc_text, doc_vocab, code_vocab, device, max_len=80, is_attention=False):
    """Generate code from docstring"""
    model.eval()

    with torch.no_grad():
        # Tokenize input
        tokens = doc_text.split()
        indices = [doc_vocab.word2idx.get(w, doc_vocab.UNK_token) for w in tokens]
        src = torch.LongTensor(indices).unsqueeze(0).to(device)
        src_lengths = torch.LongTensor([len(indices)])

        # Create initial target (SOS token)
        tgt = torch.LongTensor([[code_vocab.SOS_token]]).to(device)

        # Encode
        if is_attention:
            # Create mask
            mask = torch.ones(1, src.size(1)).to(device)
            encoder_outputs, hidden = model.encoder(src, src_lengths)
            attentions = []
        else:
            _, hidden = model.encoder(src, src_lengths)

        # Decode
        generated = [code_vocab.SOS_token]

        for _ in range(max_len):
            input_token = torch.LongTensor([[generated[-1]]]).to(device)

            if is_attention:
                output, hidden, attn_weights = model.decoder(
                    input_token, hidden, encoder_outputs, mask
                )
                attentions.append(attn_weights.cpu().numpy())
            else:
                output, hidden = model.decoder(input_token, hidden)

            predicted = output.argmax(2).item()
            generated.append(predicted)

            if predicted == code_vocab.EOS_token:
                break

        # Convert indices to words
        code_tokens = [code_vocab.idx2word[idx] for idx in generated[1:-1]]  # Skip SOS and EOS
        code_text = ' '.join(code_tokens)

        if is_attention:
            return code_text, np.array(attentions).squeeze()
        return code_text, None

def calculate_bleu(model, dataloader, doc_vocab, code_vocab, device, is_attention=False):
    """Calculate BLEU score"""
    bleu = BLEU()
    references = []
    hypotheses = []

    for batch in tqdm(dataloader, desc="Calculating BLEU"):
        doc_texts = batch['doc_texts']
        code_texts = batch['code_texts']

        for doc_text, ref_code in zip(doc_texts, code_texts):
            generated_code, _ = generate_code(
                model, doc_text, doc_vocab, code_vocab, device, is_attention=is_attention
            )
            references.append([ref_code])
            hypotheses.append(generated_code)

    score = bleu.corpus_score(hypotheses, references)
    return score.score

def calculate_exact_match(model, dataloader, doc_vocab, code_vocab, device, is_attention=False):
    """Calculate exact match accuracy"""
    correct = 0
    total = 0

    for batch in tqdm(dataloader, desc="Calculating Exact Match"):
        doc_texts = batch['doc_texts']
        code_texts = batch['code_texts']

        for doc_text, ref_code in zip(doc_texts, code_texts):
            generated_code, _ = generate_code(
                model, doc_text, doc_vocab, code_vocab, device, is_attention=is_attention
            )
            if generated_code.strip() == ref_code.strip():
                correct += 1
            total += 1

    return 100.0 * correct / total

print("Evaluation functions defined!")

In [ ]:
# Evaluate all models on test set
print("\n" + "="*60)
print("EVALUATION RESULTS ON TEST SET")
print("="*60)

evaluation_results = {}

for name, results in all_results.items():
    print(f"\nEvaluating {name}...")
    model = results['model']
    is_attention = 'Attention' in name

    # Load best model
    model_file = name.lower().replace(' + ', '_').replace(' ', '_') + '_best.pt'
    model.load_state_dict(torch.load(model_file))

    # Calculate metrics
    bleu_score = calculate_bleu(model, test_loader, doc_vocab, code_vocab,
                                device, is_attention=is_attention)
    exact_match = calculate_exact_match(model, test_loader, doc_vocab, code_vocab,
                                       device, is_attention=is_attention)

    evaluation_results[name] = {
        'BLEU': bleu_score,
        'Exact Match': exact_match
    }

    print(f"  BLEU Score: {bleu_score:.2f}")
    print(f"  Exact Match: {exact_match:.2f}%")

print("\n" + "="*60)

In [ ]:
# Create comparison table
import pandas as pd

results_df = pd.DataFrame(evaluation_results).T
print("\nComparison Table:")
print(results_df.to_string())

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# BLEU scores
models = list(evaluation_results.keys())
bleu_scores = [evaluation_results[m]['BLEU'] for m in models]
axes[0].bar(models, bleu_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[0].set_ylabel('BLEU Score')
axes[0].set_title('BLEU Score Comparison')
axes[0].set_ylim([0, max(bleu_scores) * 1.2])
for i, v in enumerate(bleu_scores):
    axes[0].text(i, v + 0.5, f"{v:.2f}", ha='center', fontweight='bold')

# Exact match
exact_matches = [evaluation_results[m]['Exact Match'] for m in models]
axes[1].bar(models, exact_matches, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1].set_ylabel('Exact Match (%)')
axes[1].set_title('Exact Match Accuracy Comparison')
axes[1].set_ylim([0, max(exact_matches) * 1.2 if max(exact_matches) > 0 else 10])
for i, v in enumerate(exact_matches):
    axes[1].text(i, v + 0.5, f"{v:.2f}%", ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('evaluation_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nEvaluation metrics chart saved as 'evaluation_metrics.png'")

## 8. Sample Generations and Error Analysis

In [ ]:
# Generate samples from test set
def show_sample_generations(models_dict, test_data, doc_vocab, code_vocab, device, n_samples=5):
    """Show sample generations from all models"""

    samples = random.sample(test_data, n_samples)

    for i, sample in enumerate(samples):
        print(f"\n{'='*80}")
        print(f"SAMPLE {i+1}")
        print(f"{'='*80}")
        print(f"\nDocstring: {sample['docstring']}")
        print(f"\nReference Code: {sample['code']}")
        print("\n" + "-"*80)

        for model_name, results in models_dict.items():
            model = results['model']
            is_attention = 'Attention' in model_name

            generated, _ = generate_code(
                model, sample['docstring'], doc_vocab, code_vocab,
                device, is_attention=is_attention
            )

            print(f"\n{model_name}:")
            print(f"  {generated}")

        print()

# Show sample generations
show_sample_generations(all_results, test_data, doc_vocab, code_vocab, device, n_samples=5)

## 9. Attention Visualization

In [ ]:
def visualize_attention(doc_text, generated_code, attention_weights,
                       doc_vocab, code_vocab, save_path=None):
    """Visualize attention heatmap"""

    # Get tokens
    doc_tokens = doc_text.split()
    code_tokens = generated_code.split()

    # Trim attention to match generated length
    attention_weights = attention_weights[:len(code_tokens), :len(doc_tokens)]

    # Create heatmap
    fig, ax = plt.subplots(figsize=(max(10, len(doc_tokens) * 0.5),
                                    max(8, len(code_tokens) * 0.4)))

    sns.heatmap(attention_weights,
                xticklabels=doc_tokens,
                yticklabels=code_tokens,
                cmap='YlOrRd',
                ax=ax,
                cbar_kws={'label': 'Attention Weight'})

    ax.set_xlabel('Docstring Tokens', fontsize=12)
    ax.set_ylabel('Generated Code Tokens', fontsize=12)
    ax.set_title('Attention Alignment Heatmap', fontsize=14, fontweight='bold')

    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

    plt.show()

# Generate attention visualizations
print("Generating attention visualizations...\n")

# Get attention model
attn_model = all_results['LSTM + Attention']['model']
attn_model.load_state_dict(torch.load('lstm_attention_best.pt'))

# Select interesting test examples
visualization_samples = random.sample(test_data, 3)

for i, sample in enumerate(visualization_samples):
    print(f"\nExample {i+1}:")
    print(f"Docstring: {sample['docstring']}")
    print(f"Reference: {sample['code']}")

    # Generate with attention
    generated_code, attention_weights = generate_code(
        attn_model, sample['docstring'], doc_vocab, code_vocab,
        device, is_attention=True
    )

    print(f"Generated: {generated_code}")

    # Visualize
    if attention_weights is not None and len(attention_weights) > 0:
        visualize_attention(
            sample['docstring'], generated_code, attention_weights,
            doc_vocab, code_vocab,
            save_path=f'attention_heatmap_{i+1}.png'
        )
        print(f"Saved attention heatmap as 'attention_heatmap_{i+1}.png'")
    print("\n" + "="*80)

## 10. Analysis: Performance vs Docstring Length

In [ ]:
def analyze_length_performance(model, test_data, doc_vocab, code_vocab, device,
                              is_attention=False, model_name="Model"):
    """Analyze performance vs docstring length"""

    # Group by length
    length_buckets = {}

    for sample in tqdm(test_data[:200], desc=f"Analyzing {model_name}"):  # Use subset for speed
        doc_len = len(sample['docstring'].split())
        bucket = (doc_len // 10) * 10  # Group by 10s

        if bucket not in length_buckets:
            length_buckets[bucket] = {'correct': 0, 'total': 0}

        generated, _ = generate_code(
            model, sample['docstring'], doc_vocab, code_vocab,
            device, is_attention=is_attention
        )

        # Check if correct
        if generated.strip() == sample['code'].strip():
            length_buckets[bucket]['correct'] += 1
        length_buckets[bucket]['total'] += 1

    # Calculate accuracy per bucket
    lengths = sorted(length_buckets.keys())
    accuracies = []

    for length in lengths:
        bucket = length_buckets[length]
        acc = 100.0 * bucket['correct'] / bucket['total'] if bucket['total'] > 0 else 0
        accuracies.append(acc)

    return lengths, accuracies

# Analyze all models
print("Analyzing performance vs docstring length...\n")

plt.figure(figsize=(12, 6))

for name, results in all_results.items():
    model = results['model']
    is_attention = 'Attention' in name

    # Load best model
    model_file = name.lower().replace(' + ', '_').replace(' ', '_') + '_best.pt'
    model.load_state_dict(torch.load(model_file))

    lengths, accuracies = analyze_length_performance(
        model, test_data, doc_vocab, code_vocab, device,
        is_attention=is_attention, model_name=name
    )

    plt.plot(lengths, accuracies, marker='o', label=name, linewidth=2)

plt.xlabel('Docstring Length (tokens)', fontsize=12)
plt.ylabel('Exact Match Accuracy (%)', fontsize=12)
plt.title('Performance vs Docstring Length', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('length_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nLength analysis saved as 'length_analysis.png'")

## 11. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("SUMMARY OF FINDINGS")
print("="*80)

print("\n1. TRAINING PERFORMANCE:")
for name, results in all_results.items():
    final_train_loss = results['train_losses'][-1]
    final_val_loss = results['val_losses'][-1]
    print(f"\n{name}:")
    print(f"  Final Training Loss: {final_train_loss:.4f}")
    print(f"  Final Validation Loss: {final_val_loss:.4f}")

print("\n\n2. TEST SET METRICS:")
print(results_df.to_string())

print("\n\n3. KEY OBSERVATIONS:")
print("\n- Vanilla RNN: ")
print("  Suffers from vanishing gradients, struggles with longer sequences")
print("\n- LSTM: ")
print("  Better at capturing long-range dependencies, improved performance")
print("\n- LSTM + Attention: ")
print("  Removes fixed-context bottleneck, best overall performance")
print("  Attention weights show semantic alignment between docstring and code")

print("\n\n4. DELIVERABLES GENERATED:")
print("  ✓ training_curves.png - Training/validation loss comparison")
print("  ✓ evaluation_metrics.png - BLEU and Exact Match comparison")
print("  ✓ attention_heatmap_1.png, 2.png, 3.png - Attention visualizations")
print("  ✓ length_analysis.png - Performance vs docstring length")
print("  ✓ vanilla_rnn_best.pt, lstm_best.pt, lstm_attention_best.pt - Saved models")

print("\n" + "="*80)
print("Assignment Complete!")
print("="*80)

## 12. Save Results for Report

In [ ]:
# Save all results to JSON for easy report generation
import json

report_data = {
    'training_config': {
        'embedding_dim': EMBED_DIM,
        'hidden_dim': HIDDEN_DIM,
        'n_epochs': N_EPOCHS,
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'train_samples': len(train_data),
        'val_samples': len(val_data),
        'test_samples': len(test_data)
    },
    'results': {}
}

for name, results in all_results.items():
    report_data['results'][name] = {
        'train_losses': results['train_losses'],
        'val_losses': results['val_losses'],
        'final_train_loss': results['train_losses'][-1],
        'final_val_loss': results['val_losses'][-1],
        'bleu_score': evaluation_results[name]['BLEU'],
        'exact_match': evaluation_results[name]['Exact Match']
    }

with open('experiment_results.json', 'w') as f:
    json.dump(report_data, f, indent=2)

print("Results saved to 'experiment_results.json'")
print("\nAll files ready for your report!")